In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

/home/thorsten/projects/private_projects/llm_rag/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [14]:
v = embedding_model.encode(user_question)

In [4]:
round(7.82226548e-02,3)

0.078

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
filtered_data = [entry for entry in documents if entry['course'] == 'machine-learning-zoomcamp']

In [8]:
len(filtered_data)

375

In [9]:
embeddings = []
for doc in filtered_data:
    qa_text = f"{doc['question']} {doc['text']}"
    res = embedding_model.encode(qa_text)
    embeddings.append(res)

In [11]:
import numpy as np

In [12]:
X = np.array(embeddings)
X.shape

(375, 768)

In [15]:
scores = X.dot(v)

In [17]:
print(max(scores))

0.6506573


In [25]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_data, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [26]:
def hit(search_engine, ground_truth, num_results=5):
    hits = 0
    for item in ground_truth:
        query = item['question']
        exp = item['document']
        query_embedding = embedding_model.encode(query)
        results = search_engine.search(query_embedding, num_results)
        res_id = [doc['id'] for doc in results]
        if exp in res_id:
            hits += 1
    return hits / len(ground_truth)

result = hit(search_engine, ground_truth, num_results=5)
print(result)

0.9398907103825137


In [27]:
print(ground_truth[:5])

[{'question': 'Where can I sign up for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Can you provide a link to sign up?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Is there an FAQ for this Machine Learning course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Does this course have a GitHub repository for the sign-up link?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'How can I structure my questions and answers for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}]


In [29]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'f501130a539b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'K-4cfUvCRyaqlUpbT2M-bw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [34]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "question_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

# Create the index
index_name = "llm-zoomcamp-index"
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-zoomcamp-index'})

In [35]:
filtered_data[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}